Data Cleaning

Feature Engineering and Data preprocessing for Train Dataset 

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Define a custom LabelEncoder class to handle unseen labels
class SafeLabelEncoder(LabelEncoder):
    def fit(self, y):
        # Fit the encoder on all the unique labels (to ensure the encoder is trained)
        super().fit(y)
        # Add an 'Unknown' label to handle unseen labels during transformation
        self.classes_ = list(self.classes_) + ['Unknown']
        return self

    def transform(self, y):
        # Transform with a fallback to 'Unknown' for unseen labels
        return [self.classes_.index(val) if val in self.classes_ else self.classes_.index('Unknown') for val in y]

# Define file paths
input_file = r'C:\Projects\Microsoft-project\GUIDE_Train.csv\GUIDE_Train.csv'
output_file = r'C:\Projects\Microsoft-project\Final_Train_DS.csv'  # Updated output file for the processed dataset

# Specify columns with high missing values to drop
cols_to_drop = [
    "ActionGrouped", "ActionGranular", "EmailClusterId", "ThreatFamily",
    "ResourceType", "Roles", "AntispamDirection", "SuspicionLevel", "LastVerdict"
]

# Columns for label encoding
categorical_columns = ["MitreTechniques", "IncidentGrade", "Category", "EntityType", "EvidenceRole"]

# Initialize label encoders
label_encoders = {col: SafeLabelEncoder() for col in categorical_columns}

# Flag to track if any integer conversion was done
conversion_successful = False

# Processing and saving data in chunks
with open(output_file, 'w') as f_out:
    for chunk in pd.read_csv(input_file, chunksize=100000, dtype={35: str}, low_memory=False):
        # Drop unnecessary columns
        chunk = chunk.drop(columns=cols_to_drop, errors='ignore')
        
        # Convert 'int64' columns to 'int32' where applicable
        for col in chunk.select_dtypes(include=['int64']).columns:
            if chunk[col].min() >= -2147483648 and chunk[col].max() <= 2147483647:
                chunk[col] = chunk[col].astype('int32')
                conversion_successful = True
        
        # Fill missing values in categorical columns with 'Unknown'
        chunk['MitreTechniques'] = chunk['MitreTechniques'].fillna('Unknown')
        chunk['IncidentGrade'] = chunk['IncidentGrade'].fillna('Unknown')

        # Fit label encoders on each categorical column
        for col in categorical_columns:
            chunk[col] = chunk[col].fillna('Unknown')
            label_encoders[col].fit(chunk[col].astype(str))

        # Convert 'Timestamp' column to datetime and extract date features
        chunk['Timestamp'] = pd.to_datetime(chunk['Timestamp'], errors='coerce')
        chunk['year'] = chunk['Timestamp'].dt.year
        chunk['month'] = chunk['Timestamp'].dt.month
        chunk['day'] = chunk['Timestamp'].dt.day
        chunk['hour'] = chunk['Timestamp'].dt.hour
        chunk['weekday'] = chunk['Timestamp'].dt.weekday
        chunk = chunk.drop(columns=['Timestamp'], errors='ignore')
        
        # Apply label encoding to categorical columns
        for col in categorical_columns:
            chunk[col] = label_encoders[col].transform(chunk[col].astype(str))
        
        # Append processed chunk to output CSV
        chunk.to_csv(f_out, index=False, header=f_out.tell()==0)

# Print messages for status
if conversion_successful:
    print("Conversion to 'int32' was successful where applicable.")
else:
    print("No columns were converted to 'int32'. All columns are either within 'int64' range or already 'int32'.")
print(f"Processed dataset saved as: {output_file}")


Conversion to 'int32' was successful where applicable.
Processed dataset saved as: C:\Projects\Microsoft-project\Final_Train_DS.csv


In [1]:
import pandas as pd
df1 = pd.read_csv("C:\Projects\Microsoft-project\Final_Train_DS.csv")
df1.head(10)

,Id,OrgId,IncidentId,AlertId,DetectorId,AlertTitle,Category,MitreTechniques,IncidentGrade,EntityType,...,OSFamily,OSVersion,CountryCode,State,City,year,month,day,hour,weekday
0,180388628218,0,612,123247,7,6,10,505,2,11,...,5,66,31,6,3,2024,6,4,6,1
1,455266534868,88,326,210035,58,43,7,505,1,24,...,5,66,242,1445,10630,2024,6,14,3,4
2,1056561957389,809,58352,712507,423,298,10,358,1,23,...,5,66,242,1445,10630,2024,6,13,4,3
3,1279900258736,92,32992,774301,2,2,1,505,0,23,...,5,66,242,1445,10630,2024,6,10,16,0
4,214748368522,148,4359,188041,9,74,6,505,2,24,...,5,66,242,1445,10630,2024,6,15,1,5
5,1322849927433,11,417400,825450,0,0,10,285,1,11,...,5,66,8,6,3,2024,6,10,13,0
6,163208760309,522,566,705663,2,2,1,505,0,23,...,5,66,242,1445,10630,2024,6,14,23,4
7,1400159339557,125,38679,47423,313,3919,7,505,0,14,...,5,66,242,1445,10630,2024,6,6,13,3
8,1219770713645,21,414,197969,3,4,16,505,0,19,...,5,66,242,1445,10630,2024,6,9,10,6
9,1073741827836,72,70,831157,4,3,10,505,2,24,...,5,66,242,1445,10630,2024,6,8,2,5


Test Dataset

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Define a custom LabelEncoder class to handle unseen labels
class SafeLabelEncoder(LabelEncoder):
    def fit(self, y):
        # Fit the encoder on all the unique labels (to ensure the encoder is trained)
        super().fit(y)
        # Add an 'Unknown' label to handle unseen labels during transformation
        self.classes_ = list(self.classes_) + ['Unknown']
        return self

    def transform(self, y):
        # Transform with a fallback to 'Unknown' for unseen labels
        return [self.classes_.index(val) if val in self.classes_ else self.classes_.index('Unknown') for val in y]

# Define file paths
input_file = r'C:\Projects\Microsoft-project\GUIDE_Test.csv\GUIDE_Test.csv'
output_file = r'C:\Projects\Microsoft-project\Final_Test_DS.csv'  # Updated output file for the processed dataset

# Specify columns with high missing values to drop
cols_to_drop = [
    "ActionGrouped", "ActionGranular", "EmailClusterId", "ThreatFamily",
    "ResourceType", "Roles", "AntispamDirection", "SuspicionLevel", "LastVerdict"
]

# Columns for label encoding
categorical_columns = ["MitreTechniques", "IncidentGrade", "Category", "EntityType", "EvidenceRole"]

# Initialize label encoders
label_encoders = {col: SafeLabelEncoder() for col in categorical_columns}

# Flag to track if any integer conversion was done
conversion_successful = False

# Processing and saving data in chunks
with open(output_file, 'w') as f_out:
    for chunk in pd.read_csv(input_file, chunksize=100000, dtype={35: str}, low_memory=False):
        # Drop unnecessary columns
        chunk = chunk.drop(columns=cols_to_drop, errors='ignore')
        
        # Convert 'int64' columns to 'int32' where applicable
        for col in chunk.select_dtypes(include=['int64']).columns:
            if chunk[col].min() >= -2147483648 and chunk[col].max() <= 2147483647:
                chunk[col] = chunk[col].astype('int32')
                conversion_successful = True
        
        # Fill missing values in categorical columns with 'Unknown'
        chunk['MitreTechniques'] = chunk['MitreTechniques'].fillna('Unknown')
        chunk['IncidentGrade'] = chunk['IncidentGrade'].fillna('Unknown')

        # Fit label encoders on each categorical column
        for col in categorical_columns:
            chunk[col] = chunk[col].fillna('Unknown')
            label_encoders[col].fit(chunk[col].astype(str))

        # Convert 'Timestamp' column to datetime and extract date features
        chunk['Timestamp'] = pd.to_datetime(chunk['Timestamp'], errors='coerce')
        chunk['year'] = chunk['Timestamp'].dt.year
        chunk['month'] = chunk['Timestamp'].dt.month
        chunk['day'] = chunk['Timestamp'].dt.day
        chunk['hour'] = chunk['Timestamp'].dt.hour
        chunk['weekday'] = chunk['Timestamp'].dt.weekday
        chunk = chunk.drop(columns=['Timestamp'], errors='ignore')
        
        # Apply label encoding to categorical columns
        for col in categorical_columns:
            chunk[col] = label_encoders[col].transform(chunk[col].astype(str))
        
        # Append processed chunk to output CSV
        chunk.to_csv(f_out, index=False, header=f_out.tell()==0)

# Print messages for status
if conversion_successful:
    print("Conversion to 'int32' was successful where applicable.")
else:
    print("No columns were converted to 'int32'. All columns are either within 'int64' range or already 'int32'.")
print(f"Processed dataset saved as: {output_file}")


Conversion to 'int32' was successful where applicable.
Processed dataset saved as: C:\Projects\Microsoft-project\Final_Test_DS.csv


In [4]:
import pandas as pd

# Load the data
df1 = pd.read_csv(r"C:\Projects\Microsoft-project\Final_Train_DS.csv")

# Select the first 10 rows
df1_sample = df1.head(10)

# Save to CSV
df1_sample.to_csv(r"C:\Projects\Microsoft-project\Final_Train_DS_sample.csv", index=False)
